In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
import unicodedata

from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.palettes import Category20, Viridis256, RdYlGn
from bokeh.models import GeoJSONDataSource, ColumnDataSource, HoverTool, Legend, TabPanel, Tabs, ColorBar
from bokeh.transform import linear_cmap, LinearColorMapper

In [ ]:
database = pd.read_csv('Vítimas de Homicidio Consumado - Jan 2012 a Fev 2024.csv', encoding='latin_1', sep=';')

In [ ]:
database

In [ ]:
database = database.drop(columns=['Número REDS','Número Envolvido/Ocorrência','Natureza Principal Final','Horário Fato','Relação Vítima/Autor','Tipo Logradouro Envolvido', 'BAIRRO - FATO FINAL', 'RMBH',
                      'Faixa 6 Horas Fato','Descrição Grupo Local Imediato','Tipo_Envolvimento_Lesão_Final','Bairro Envolvido', 'Bairro Envolvido Não Cadastrado', 'Logradouro Ocorrência - Tipo',
                      'Município Envolvido','UF Envolvido - Nome', 'RISP', 'UF - Sigla - FATO', 'Município - Código - FATO', 'BAIRRO - FATO FINAL -Município', 'Bairro Não Cadastrado - FATO',
                      'Bairro - FATO'])
database['Data Numerica'] = database['Ano Fato'] + (database['Mês Numérico Fato'] - 1)/12
database

In [ ]:
output_notebook()

# Quem é vitimizado?

Perfil racial

In [ ]:
db = database[['Ano Fato','Cútis']]

# group by 'Year' and 'Category' and count the occurrences
category_counts = db.groupby(['Ano Fato', 'Cútis']).size().reset_index(name='Count')
total = category_counts.groupby('Ano Fato')['Count'].transform('sum')
category_counts['Count'] = category_counts['Count'] / total

# pivot to have 'Category' as columns
pivoted = category_counts.pivot(index='Ano Fato', columns='Cútis', values='Count').fillna(0)

pivoted['NÃO INFORMADA'] = pivoted['DESCONHECIDA'] + pivoted['DESCONHECIDO'] + pivoted['IGNORADA'] + pivoted['PREENCHIMENTO OPCIONAL']
pivoted = pivoted.drop(columns=['DESCONHECIDA', 'DESCONHECIDO', 'IGNORADA', 'PREENCHIMENTO OPCIONAL'])

# get the list of categories
categories = pivoted.columns

In [ ]:
# set up figure
p1 = figure(x_axis_label='Ano', y_axis_label='Fração dos mortos', height=800, width=1200, y_range=[0,1], x_range=(2012,2024), tools='save', sizing_mode='stretch_width')

# add stack
stack = p1.varea_stack(categories, x='Ano Fato', source=pivoted, color=Category20[len(categories)])

# set legends
legend = Legend(items=[(x, [stack[i]]) for i, x in enumerate(categories)])
p1.add_layout(legend, 'right')
p1.legend.orientation = 'vertical'

# change style of title
p1.title = "Perfil Demográfico dos Homicídios em Minas Gerais (2012 - Fev 2024)"
p1.title.text_font_size = "24px"
p1.title.text_color = "navy"

# save the plot
tab1 = TabPanel(child=p1, title='Cútis')

Nível de Escolaridade

In [ ]:
db = database[['Ano Fato','Escolaridade']]

# group by 'Year' and 'Category' and count the occurrences
category_counts = db.groupby(['Ano Fato', 'Escolaridade']).size().reset_index(name='Count')
total = category_counts.groupby('Ano Fato')['Count'].transform('sum')
category_counts['Count'] = category_counts['Count'] / total

# pivot to have 'Category' as columns
pivoted = category_counts.pivot(index='Ano Fato', columns='Escolaridade', values='Count').fillna(0)

# get the list of unique categories
categories = pivoted.columns

In [ ]:
# set up figure
p2 = figure(x_axis_label='Ano', y_axis_label='Fração dos mortos', height=800, width=1200, y_range=[0,1], x_range=(2012,2024), tools='save', sizing_mode='stretch_width')

# add stack
stack = p2.varea_stack(categories, x='Ano Fato', source=pivoted, color=Category20[len(categories)])

# set legends
legend = Legend(items=[(x, [stack[i]]) for i, x in enumerate(categories)])
p2.add_layout(legend, 'right')
p2.legend.orientation = 'vertical'

# change style of title
p2.title = "Perfil Demográfico dos Homicídios em Minas Gerais (2012 - Fev 2024)"
p2.title.text_font_size = "24px"
p2.title.text_color = "navy"

# save the plot
tab2 = TabPanel(child=p2, title='Escolaridade')

Faixa Etária

In [ ]:
db = database[['Ano Fato','Idade Aparente']].copy()

# convert valid age values to numeric
db['Idade Aparente'] = pd.to_numeric(db['Idade Aparente'], errors='coerce')

# categorize 'Idade Aparente'
bins = [0, 12, 18, 24, 30, 36, 42, 48, 54, 60, 72, 100]
labels = ['0-12', '13-18', '19-24', '25-30', '31-36', '37-42', '43-48', '49-54', '55-60', '60-72', 'up to 100']
db['Faixa Etaria'] = pd.cut(db['Idade Aparente'], bins=bins, labels=labels, right=False)
db = db.drop(columns=['Idade Aparente'])

# handle 'DESCONHECIDA' values separately
db['Faixa Etaria'] = db['Faixa Etaria'].cat.add_categories(['DESCONHECIDA']).fillna('DESCONHECIDA')

# group by 'Year' and 'Category' and count the occurrences
category_counts = db.groupby(['Ano Fato', 'Faixa Etaria']).size().reset_index(name='Count')
total = category_counts.groupby('Ano Fato')['Count'].transform('sum')
category_counts['Count'] = category_counts['Count'] / total

# pivot to have 'Category' as columns
pivoted = category_counts.pivot(index='Ano Fato', columns='Faixa Etaria', values='Count').fillna(0)

# get the list of unique categories
categories = pivoted.columns

In [ ]:
# set up figure
p3 = figure(x_axis_label='Ano', y_axis_label='Fração dos mortos', height=800, width=1200, y_range=[0,1], x_range=(2012,2024), tools='save', sizing_mode='stretch_width')

# add stack
stack = p3.varea_stack(categories, x='Ano Fato', source=pivoted, color=Category20[len(categories)])

# set legends
legend = Legend(items=[(x, [stack[i]]) for i, x in enumerate(categories)], title="Faixa Etaria")
p3.add_layout(legend, 'right')
p3.legend.orientation = 'vertical'

# change style of title
p3.title = "Perfil Demográfico dos Homicídios em Minas Gerais (2012 - Fev 2024)"
p3.title.text_font_size = "24px"
p3.title.text_color = "navy"

# save the plot
tab3 = TabPanel(child=p3, title='Idade')

Sexo

In [ ]:
db = database[['Ano Fato','Sexo']].copy()

# group by 'Year' and 'Category' and count the occurrences
category_counts = db.groupby(['Ano Fato', 'Sexo']).size().reset_index(name='Count')
total = category_counts.groupby('Ano Fato')['Count'].transform('sum')
category_counts['Count'] = category_counts['Count'] / total

# pivot to have 'Category' as columns
pivoted = category_counts.pivot(index='Ano Fato', columns='Sexo', values='Count').fillna(0)

# remove non-binary data as it is less relevant
pivoted = pivoted.drop(columns=['DESCONHECIDO', 'NÃO IDENTIFICADO', 'NÃO INFORMADO'])
pivoted["NÃO INFORMADO"] = 1 - pivoted['FEMININO'] - pivoted['MASCULINO']

# get the list of unique categories
categories = pivoted.columns

In [ ]:
# set up figure
p4 = figure(x_axis_label='Ano', y_axis_label='Fração dos mortos', height=800, width=2200, y_range=[0,1], x_range=(2012,2024), tools='save', sizing_mode='fixed')

# add stack
stack = p4.varea_stack(categories, x='Ano Fato', source=pivoted, color=['firebrick', 'navy', 'green'])

# set legends
legend = Legend(items=[(x, [stack[i]]) for i, x in enumerate(categories)], title="Sexo")
p4.add_layout(legend, 'right')
p4.legend.orientation = 'vertical'

# change style of title
p4.title = "Perfil Demográfico dos Homicídios em Minas Gerais (2012 - Fev 2024)"
p4.title.text_font_size = "24px"
p4.title.text_color = "navy"

# save the plot
tab4 = TabPanel(child=p4, title='Sexo')

Combined Plots

In [ ]:
tabs = Tabs(tabs=[tab1, tab2, tab3, tab4])
show(tabs)

# Onde ocorrem?

In [ ]:
# read municipality map found in 'https://geoftp.ibge.gov.br/organizacao_do_territorio/malhas_territoriais/malhas_municipais/municipio_2021/Brasil/BR/BR_Municipios_2021.zip'
mapa = gpd.read_file('zip:municipios.zip')
mapa = mapa[(mapa['SIGLA'] == 'MG')]

In [ ]:
# Find the relative number of homicides for municipality
relative = database['Município - FATO'].value_counts().reset_index()
relative.columns = ['NM_MUN', 'count']
relative['percentile'] = relative['count'] * 100 / len(database)
relative['log'] = np.log(relative['count'] + 1) 

In [ ]:
def normalize_name(text):
    nfkd_form = unicodedata.normalize('NFKD', text)
    only_ascii = nfkd_form.encode('ASCII', 'ignore').decode('ASCII')
    return only_ascii.upper()
    
# normalize municipality names
relative['NM_MUN'] = relative['NM_MUN'].apply(normalize_name)
mapa['NM_MUN'] = mapa['NM_MUN'].apply(normalize_name)

# join map and data
mapa = mapa.set_index('NM_MUN').join(relative.set_index('NM_MUN'), how='left').fillna(0)
mapa = mapa.reset_index()

# create geodata
geo_source = GeoJSONDataSource(geojson=mapa.to_json())

In [ ]:
# set up the tooltips
TOOLTIPS = [
    ("Município", "@NM_MUN"),
    ("Área", "@AREA_KM2{0.1} km²"),
    ("# de Homicídios", "@count{(0,0)}"),
    ("Porcentagem", "@percentile{(2.2)} %"),
]

# set up the figure
map_plot = figure(
    height=200,  # set a width and height to define the aspect ratio
    width=300,
    sizing_mode="scale_width",
    tooltips=TOOLTIPS,
    title="Homicídios no Estado de Minas Gerais (2012 - Fev 2024)",
    x_axis_location=None,  # deactivate x-axis
    y_axis_location=None,  # deactivate y-axis
    tools="save,hover",    # deactivate useless tools
)
map_plot.grid.grid_line_color = None  # make grid lines invisible

# draw the state polygons
mg = map_plot.patches(  # use the patches method to draw the polygons of all states
    xs="xs",
    ys="ys",
    fill_color=linear_cmap(field_name="log", palette=Viridis256, low=mapa["log"].min(), high=mapa["log"].max()),  # color the states by mapping the number of routes to color values from a palette
    source=geo_source,
    line_color="black",
    line_width=1,
)

# add color bar
color_bar = mg.construct_color_bar(height=10, title="Logaritmo de 2 do total de mortos")
map_plot.add_layout(obj=color_bar, place="below")

# change style of title
map_plot.title.text_font_size = "24px"
map_plot.title.text_color = "navy"

show(map_plot)

# Quando ocorrem?

### Evolução anual

In [ ]:
# Remove the incomplete 2024 to avoid biases on the monthly analyze
db = database[database['Ano Fato'] != 2024]

# Find the relative number of homicides for each month
relative = db['Ano Fato'].value_counts()

# Order the index
year = [2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]
ordered = relative.reindex(year)

In [ ]:
# set up figure
p = figure(title="Evolução no número de homicidios no estado", x_axis_label='Ano', y_axis_label='Total de mortos', width=1200, tools="save")

x = year
y = ordered.values

# add line
p.line(x=x, y=y, color='navy', width=10)

# add change bars
source = ColumnDataSource(data=dict(x=x[1:], bottom=y[:-1], top=y[1:], percentile=np.diff(y)/y[:-1] * 100))
mapper = linear_cmap(field_name='percentile', palette=RdYlGn[10], low=-25, high=25)
p.vbar(x='x', bottom='bottom', top='top', width=.1, color=mapper, source=source)

# add color bar
color_mapper = LinearColorMapper(palette=RdYlGn[10], low=-25, high=25)
color_bar = ColorBar(color_mapper=color_mapper, label_standoff=25, location=(0,0), title='Variação Percentual de Ano a Ano')
p.add_layout(color_bar, 'right')

# change style of title
p.title.text_font_size = "24px"
p.title.text_color = "navy"

# remove grid lines
p.xgrid.grid_line_color = None
p.axis.minor_tick_line_color = None

# show the plot
show(p)

### Distribuição

Monthly

In [ ]:
# remove the incomplete 2024 to avoid biases on the monthly analyze
db = database[database['Ano Fato'] != 2024]

# find the relative number of homicides for each month
relative = db['Mês Fato Resumido'].value_counts() / len(db) * 100

# order the index
year = ['JAN', 'FEV', 'MAR', 'ABR', 'MAI', 'JUN', 'JUL', 'AGO', 'SET', 'OUT', 'NOV', 'DEZ']
ordered = relative.reindex(year)

In [ ]:
# set up figure
p = figure(x_axis_label='Mês', y_axis_label='Porcentagem', x_range=year, width=1200, tools="save")

# add bars
p.vbar(x=year, top=ordered.values, width=0.95, color='navy')

# add a line to show what an equal distribution would look like
p.line(x=[0,12], y=[100/12, 100/12], color='red', width=5)

# change style of title
p.title = "Distribuição dos Homicídios em Minas Gerais (2012 - 2023)"
p.title.text_font_size = "24px"
p.title.text_color = "navy"

# remove grid lines
p.xgrid.grid_line_color = None

# save the plot
tab1 = TabPanel(child=p, title='Mensal')

Weekly

In [ ]:
# find the relative number of homicides for each month
relative = database['Dia da Semana Fato'].value_counts() / len(database) * 100

# order the index
week = ['DOMINGO', 'SEGUNDA-FEIRA', 'TERÇA-FEIRA', 'QUARTA-FEIRA', 'QUINTA-FEIRA', 'SEXTA-FEIRA', 'SÁBADO']
ordered = relative.reindex(week)

In [ ]:
# set up figure
p = figure(x_axis_label='Dia', y_axis_label='Porcentagem', x_range=week, width=1200, tools="save")

# add bars
p.vbar(x=week, top=ordered.values, width=0.95, color='navy')

# add a line to show what an equal distribution would look like
p.line(x=[0,7], y=[100/7, 100/7], color='red', width=5)

# change style of title
p.title = "Distribuição dos Homicídios em Minas Gerais (2012 - Fev 2024)"
p.title.text_font_size = "24px"
p.title.text_color = "navy"

# remove grid lines
p.xgrid.grid_line_color = None

# save the plot
tab2 = TabPanel(child=p, title='Semanal')

Daily

In [ ]:
# find the relative number of homicides for each month
relative = database['Faixa 1 Hora Fato'].value_counts() / len(database) * 100

# order the index
ordered = relative.sort_index()

In [ ]:
# create a figure
p = figure(x_axis_label='Faixa de hora', y_axis_label='Porcentagem', x_range=(0,24), width=1200, tools="save")

# add bars
p.vbar(x=[i + 0.5 for i in range(24)], top=ordered.values, width=0.95, color='navy')

# add a line to show what an equal distribution would look like
p.line(x=[0,24], y=[100/24, 100/24], color='red', width=5)

# change style of title
p.title = "Distribuição dos Homicídios em Minas Gerais (2012 - Fev 2024)"
p.title.text_font_size = "24px"
p.title.text_color = "navy"

# remove grid lines
p.xgrid.grid_line_color = None

# save the plot
tab3 = TabPanel(child=p, title='Diaria')

In [ ]:
tabs = Tabs(tabs=[tab1, tab2, tab3])
show(tabs)

# Como ocorrem?

In [ ]:
db = database[['Ano Fato','Desc Longa Meio Utilizado']].copy()
db = db[db['Ano Fato'] != 2024]

# group by 'Year' and 'Category' and count the occurrences
category_counts = db.groupby(['Ano Fato', 'Desc Longa Meio Utilizado']).size().reset_index(name='Count')

# pivot to have 'Category' as columns
pivoted = category_counts.pivot(index='Ano Fato', columns='Desc Longa Meio Utilizado', values='Count').fillna(0)

# group redundant causes
pivoted['AGRESSÃO FISICA'] = pivoted['AGRESSAO FISICA'] + pivoted['AGRESSAO FISICA COM EMPREGO DE INSTRUMENTOS'] + pivoted['AGRESSAO FISICA SEM EMPREGO DE INSTRUMENTOS']
pivoted = pivoted.drop(columns=['AGRESSAO FISICA', 'AGRESSAO FISICA COM EMPREGO DE INSTRUMENTOS', 'AGRESSAO FISICA SEM EMPREGO DE INSTRUMENTOS'])

pivoted['ASFIXIA'] = pivoted['ASFIXIA'] + pivoted['AFOGAMENTO'] + pivoted['ASFIXIA MECANICA (ENFORCAMENTO, ESTRANGULAMENTO, ESGANADURA OU SUFOCAMENTO)'] + pivoted['ASFIXIA POR AFOGAMENTO'] + pivoted['CORDA / ENFORCAMENTO']
pivoted = pivoted.drop(columns=['AFOGAMENTO', 'ASFIXIA MECANICA (ENFORCAMENTO, ESTRANGULAMENTO, ESGANADURA OU SUFOCAMENTO)', 'ASFIXIA POR AFOGAMENTO', 'CORDA / ENFORCAMENTO'])

pivoted['ROMPIMENTO DE OBSTÁCULO'] = pivoted['ABUSO DE CONFIANCA/CONCURSO DE PESSOAS/ DESTREZA/ DESTRUICAO OU ROMPIMENTO DE OBSTACULO/ESCALADA/FRAUDE/USO DE CHAVE FALSA'] + pivoted['ARROMBAMENTO / ROMPIMENTO DE OBSTACULO']
pivoted = pivoted.drop(columns=['ABUSO DE CONFIANCA/CONCURSO DE PESSOAS/ DESTREZA/ DESTRUICAO OU ROMPIMENTO DE OBSTACULO/ESCALADA/FRAUDE/USO DE CHAVE FALSA', 'ARROMBAMENTO / ROMPIMENTO DE OBSTACULO'])

pivoted['DECAPITAÇÃO / DEGOLA / ESGORJAMENTO'] = pivoted['DECAPITAÇÃO / DEGOLA / ESGORJAMENTO'] + pivoted['DECAPITACAO / DEGOLA / ESGORJAMENTO']
pivoted.drop(columns=['DECAPITACAO / DEGOLA / ESGORJAMENTO'])

pivoted['FOGO/EXPLOSIVOS'] = pivoted['EXPLOSIVO / INFLAMAVEL'] + pivoted['INFLAMAVEIS / COMBUSTIVEIS / QUIMICOS / EXPLOSIVOS / ELETRICIDADE'] + pivoted['INFLAMAVEIS/COMBUSTIVEIS/QUIMICOS/EXPLOSIVOS/FOGO'] + pivoted['MEIO UTILIZADO - FOGO']
pivoted = pivoted.drop(columns=['EXPLOSIVO / INFLAMAVEL', 'INFLAMAVEIS / COMBUSTIVEIS / QUIMICOS / EXPLOSIVOS / ELETRICIDADE', 'INFLAMAVEIS/COMBUSTIVEIS/QUIMICOS/EXPLOSIVOS/FOGO', 'MEIO UTILIZADO - FOGO'])

pivoted['DECAPITAÇÃO / DEGOLA / ESGORJAMENTO'] = pivoted['DECAPITAÇÃO / DEGOLA / ESGORJAMENTO'] + pivoted['DECAPITACAO / DEGOLA / ESGORJAMENTO']
pivoted = pivoted.drop(columns=['DECAPITACAO / DEGOLA / ESGORJAMENTO'])

pivoted['ARMAS BRANCAS'] = pivoted['INSTRUMENTO PERFURANTE, CORTANTE OU CONTUNDENTE'] + pivoted['INSTRUMENTO CONTUNDENTE / CORTANTE / PERFURANTE (ARMA BRANCA)']
pivoted = pivoted.drop(columns=['INSTRUMENTO PERFURANTE, CORTANTE OU CONTUNDENTE', 'INSTRUMENTO CONTUNDENTE / CORTANTE / PERFURANTE (ARMA BRANCA)'])

pivoted['OUTROS OU NENHUM IDENTIFICADO'] = pivoted['INEXISTENTE'] + pivoted['INVÁLIDO'] + pivoted['MEIO UTILIZADO - IGNORADO'] + pivoted['OUTROS MEIOS (DESCREVER EM CAMPO ESPECIFICO)'] + pivoted['PREENCHIMENTO OPCIONAL']
pivoted = pivoted.drop(columns=['INEXISTENTE', 'INVÁLIDO', 'MEIO UTILIZADO - IGNORADO', 'OUTROS MEIOS (DESCREVER EM CAMPO ESPECIFICO)', 'PREENCHIMENTO OPCIONAL'])

# get the list of unique categories
categories = pivoted.columns

In [ ]:
# set up the tooltips
TOOLTIPS = [
    ("# de Homicídios", "@$name"),
]

# set up figure
p = figure(x_axis_label='Ano', y_axis_label='Total de mortos', width=1200, height=1000,x_range=(2011.5,2023.5), y_range=(0,4500), sizing_mode='stretch_width', tools='save,hover', tooltips=TOOLTIPS)

# add bars
bars = p.vbar_stack(categories, x='Ano Fato', width=0.95, source=pivoted, color=Category20[len(categories)])

# set up legends
legend = Legend(items=[(x, [bars[i]]) for i, x in enumerate(categories)])
p.add_layout(legend, 'right')
p.legend.orientation = 'vertical'

# change style of title
p.title = "Meios Utilizados nos Homicídios em Minas Gerais (2012 - 2023)"
p.title.text_font_size = "24px"
p.title.text_color = "navy"

# remove grid lines
p.xgrid.grid_line_color=None
p.axis.minor_tick_line_color = None

# show the plot
show(p)